# Enriquecimiento de los datos

In [1]:
%matplotlib inline
import pandas as pd

In [4]:
compras_df = pd.read_csv('data/clean_compras.csv')

In [6]:
compras_df.dtypes

NUM BEN           int64
BENEFICIARIO     object
RFC              object
FACTURA          object
CONTRATO         object
CONCEPTO         object
TIPO             object
# OP            float64
FECHA            object
IMPORTE         float64
SHEET            object
dtype: object

Vemos que el campo `FECHA` esta como `object`, entonces tenemos que hacer una expansión de fecha porque queremos hacer un ejemplo muy visible, la capacidad de  las variables para hacer esta expansión.

## Empezamos con una expansión de datos que tenemos

In [9]:
# Convertimos una columna a fecha.
compras_df['FECHA'] = pd.to_datetime(compras_df['FECHA'])
compras_df['FECHA']

0      2020-12-11
1      2020-12-11
2      2020-12-11
3      2020-12-11
4      2020-12-23
          ...    
1535   2020-12-11
1536   2020-12-11
1537   2020-12-16
1538   2020-12-15
1539   2020-12-11
Name: FECHA, Length: 1540, dtype: datetime64[ns]

In [10]:
compras_df.dtypes

NUM BEN                  int64
BENEFICIARIO            object
RFC                     object
FACTURA                 object
CONTRATO                object
CONCEPTO                object
TIPO                    object
# OP                   float64
FECHA           datetime64[ns]
IMPORTE                float64
SHEET                   object
dtype: object

Ahora vamos a expandir el campo de fecha a día, mes y año. Esto nos ayuda especialmente en modelos de clasificación en la cuál se puede hacer una delimitación clara cuando estamos en ciertos meses o años.

In [12]:
compras_df['DIA'] = compras_df['FECHA'].apply(lambda x: x.day) #día
compras_df['MES'] = compras_df['FECHA'].apply(lambda x: x.month) # mes
compras_df['YEAR'] = compras_df['FECHA'].apply(lambda x: x.year) # año

In [14]:
compras_df.head(5)

,NUM BEN,BENEFICIARIO,RFC,FACTURA,CONTRATO,CONCEPTO,TIPO,# OP,FECHA,IMPORTE,SHEET,DIA,MES,YEAR
0,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS AF8458-158008-3972,OT. 20204373,REEMPLAZO E INSTALACION DE AMORTIGUADORES (DEL...,TR,1019110.0,2020-12-11,14911.80,GASTOS VARIOS,11,12,2020
1,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF6916-158008-4041,OT. 20204425,REEMPLAZO E INSTALACION DE OCHO BIRLOS Y NUEVE...,TR,1019110.0,2020-12-11,2528.80,GASTOS VARIOS,11,12,2020
2,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF6917-158008-3975,OT. 20204376,REEMPLAZO DE DOS RINES (17). PATRIMONIO 80383...,TR,1019110.0,2020-12-11,8500.02,GASTOS VARIOS,11,12,2020
3,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF7004-158008-3971,OT. 20204374,REEMPLAZO DE DOS RINES (14). PATRIMONIO 8007...,TR,1019110.0,2020-12-11,2400.01,GASTOS VARIOS,11,12,2020
4,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF7137-158008-4554,OT. 20205130,REEMPLAZO E INSTALACION DE RIN 14. PATRIMONIO ...,TR,299001496.0,2020-12-23,1537.00,GASTOS VARIOS,23,12,2020


## Proyectemos unas variables contra otras

Vamps hacer equivalencias de una variable con respecto a otras

* Proyección de beneficiario por ventas

In [20]:
# Ventas por beneficiario, lo convertimos en un diccionario
beneficiario_ventas = compras_df['BENEFICIARIO'].value_counts().to_dict()
list(beneficiario_ventas.keys())[0:5]

['CAMARA MEXICANA DE LA INDUSTRIA DE LA CONSTRUCCION',
 'AUTOKAM REGIOMONTANA,S.A.DE C.V.',
 'CAR ONE MONTERREY, S.A. DE C.V.',
 'PEREZ RIOS RAMIRO',
 'S.I.M.E.P.R.O.D.E.']

Nos muestra la llave de cada una de ellas, cada llave va a tener adentro, el conteo de la cantidad de pagos que tiene disponible.

Estamos generando la cantidad de pagos por cada uno de los proveedores o beneficiarios.

In [22]:
compras_df['BENEFICIARIO_VTAS'] =  compras_df['BENEFICIARIO'].apply(lambda x: beneficiario_ventas[x])
compras_df.head(3)

,NUM BEN,BENEFICIARIO,RFC,FACTURA,CONTRATO,CONCEPTO,TIPO,# OP,FECHA,IMPORTE,SHEET,DIA,MES,YEAR,BENEFICIARIO_VTAS
0,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS AF8458-158008-3972,OT. 20204373,REEMPLAZO E INSTALACION DE AMORTIGUADORES (DEL...,TR,1019110.0,2020-12-11,14911.80,GASTOS VARIOS,11,12,2020,13
1,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF6916-158008-4041,OT. 20204425,REEMPLAZO E INSTALACION DE OCHO BIRLOS Y NUEVE...,TR,1019110.0,2020-12-11,2528.80,GASTOS VARIOS,11,12,2020,13
2,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF6917-158008-3975,OT. 20204376,REEMPLAZO DE DOS RINES (17). PATRIMONIO 80383...,TR,1019110.0,2020-12-11,8500.02,GASTOS VARIOS,11,12,2020,13


Vamos hacer lo mismo con el promedio por tipo de hoja(SHEET) o rubro. Estamos enriqueciendo una variable categórica con valores ordinales, lo estamos convirtiendo a expresiones variables contra otras, puede que esto ayude a tomar decisiones.

Con esto estamos haciendo ingenieria de datos.

In [25]:
# Promedio de cada uno de esos pagos
sheet_prom_compras = compras_df.groupby('SHEET')['IMPORTE'].mean().to_dict()
sheet_prom_compras

{'COMUNICACION': 109705.34366197183,
 'CONTRATISTAS Y FDO FED': 474566.76578729285,
 'GASTOS VARIOS': 881659.7317971016,
 'SERV PPROF': 229592.16666666666,
 'SERV. PERS.': 59623.40521739131}

In [26]:
compras_df['IMPORTE_GTO_PROM'] =  compras_df['SHEET'].apply(lambda x: sheet_prom_compras[x])
compras_df.head(3)

,NUM BEN,BENEFICIARIO,RFC,FACTURA,CONTRATO,CONCEPTO,TIPO,# OP,FECHA,IMPORTE,SHEET,DIA,MES,YEAR,BENEFICIARIO_VTAS,IMPORTE_GTO_PROM
0,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS AF8458-158008-3972,OT. 20204373,REEMPLAZO E INSTALACION DE AMORTIGUADORES (DEL...,TR,1019110.0,2020-12-11,14911.80,GASTOS VARIOS,11,12,2020,13,881659.731797
1,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF6916-158008-4041,OT. 20204425,REEMPLAZO E INSTALACION DE OCHO BIRLOS Y NUEVE...,TR,1019110.0,2020-12-11,2528.80,GASTOS VARIOS,11,12,2020,13,881659.731797
2,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF6917-158008-3975,OT. 20204376,REEMPLAZO DE DOS RINES (17). PATRIMONIO 80383...,TR,1019110.0,2020-12-11,8500.02,GASTOS VARIOS,11,12,2020,13,881659.731797


## Traernos información externa.

Enriquecer los datos usando una tool de páginas de linkeding.

Vamos a usar una herramienta que se llama `PhantomBuster` https://phantombuster.com/phantombuster?category=linkedin
Con el nomnbre del beneficiario fuimos a revisar su página de LinkedIn, para ver si encontrabamos una coincidencia, puede que con algunos este incorrecto pero podemos usarla como una fuente de información que nos puede traer información interesante.

In [27]:
#Creamos el nuevo DataFrame con todos los beneficiarios
pd.DataFrame(compras_df['BENEFICIARIO'].unique()).to_csv('data/unique_beneficiarios.csv', index=None)

Queda pendiente el Web Scraping de Linkeding

In [29]:
#Cargar
linkedin_mty = pd.read_csv('data/linkedin_mty.csv')

In [34]:
# Dtectamos cuáles tienen linkedin y cuales no.
has_site = {}
for i, ele in linkedin_mty.fillna(True).iterrows():
    has_site[ele['query']] = True if ele['error'] == True else False

list(has_site)[0:10]

['Rosa Lorena Valdez Miranda',
 'Roque Yañez Ramos',
 'Laura Perla Cordova Rodriguez',
 'Rosa Ofelia Coronado Flores',
 'Alvaro Flores Palomo',
 'Liliana Tijerina Cantu',
 'Ruben Basaldua Moyeda',
 'Maria Cristina Muñoz Rios',
 'Gerardo Guerrero Adame',
 'Mayela Maria De Lourdes Quiroga Tamez']

Vamos a genear una característica has

In [35]:
#Eliminamos los doble espacio
compras_df['HAS_LINKEDIN_SITE'] = compras_df['BENEFICIARIO'].apply(lambda x: has_site[x.replace('  ', ' ')])

In [38]:
compras_df['HAS_LINKEDIN_SITE'].value_counts()

True     1392
False     148
Name: HAS_LINKEDIN_SITE, dtype: int64

In [39]:
compras_df.to_csv('data/compras_df_enriched.csv', index=None)